In [2]:
# Import all needed libraries
#! pip install google-cloud-bigquery
from google.cloud import bigquery
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
#from config import db_password

In [3]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="client_secret.json"

In [4]:
client = bigquery.Client()

In [5]:
hn_dataset_ref = client.dataset('stackoverflow', project='bigquery-public-data')
hn_dataset_ref

DatasetReference('bigquery-public-data', 'stackoverflow')

In [6]:
type(hn_dataset_ref)

google.cloud.bigquery.dataset.DatasetReference

In [7]:
hn_dset = client.get_dataset(hn_dataset_ref)
hn_dset

Dataset(DatasetReference('bigquery-public-data', 'stackoverflow'))

In [8]:
type(hn_dset)

google.cloud.bigquery.dataset.Dataset

In [9]:
[x.table_id for x in client.list_tables(hn_dset)]

['badges',
 'comments',
 'post_history',
 'post_links',
 'posts_answers',
 'posts_moderator_nomination',
 'posts_orphaned_tag_wiki',
 'posts_privilege_wiki',
 'posts_questions',
 'posts_tag_wiki',
 'posts_tag_wiki_excerpt',
 'posts_wiki_placeholder',
 'stackoverflow_posts',
 'tags',
 'users',
 'votes']

In [1]:
# Exploring data in posts_questions by not answered v. answered

hn_full = client.get_table(table='bigquery-public-data.stackoverflow.posts_questions')
query_job = client.query(
        """
        SELECT
        id, title, body, accepted_answer_id, answer_count, creation_date, favorite_count, score, tags, view_count
        FROM `bigquery-public-data.stackoverflow.posts_questions`
        WHERE answer_count is not null AND creation_date > '2021-01-01'
        ORDER BY answer_count
        """
    )

results = query_job.result() 
results_list = [row for row in results]
cols = ['q_id', 'q_title', 'q_body', 'accepted_answer_id', 'answer_count', 'q_creation_date', 'q_favorite_count', 'q_score', 'q_tags', 'q_view_count']
posts_questions_df = pd.DataFrame.from_records(results_list, columns=cols)
posts_questions_df

NameError: name 'client' is not defined

In [50]:
# Extract data for May 2021 for posts without answers
#hn_full = client.get_table(table='bigquery-public-data.stackoverflow.posts_questions')
#query_job = client.query(
        #"""
        #SELECT
        #id, title, body, accepted_answer_id, answer_count, creation_date, favorite_count, score, tags, view_count
        #FROM `bigquery-public-data.stackoverflow.posts_questions`
        #WHERE answer_count is null AND creation_date > '2021-01-01'
        #ORDER BY answer_count
        #LIMIT 10"""
    #)

#results = query_job.result() 
#results_list = [row for row in results]
#cols = ['q_id', 'q_title', 'q_body', 'accepted_answer_id', 'answer_count', 'q_creation_date', 'q_favorite_count', 'q_score', 'q_tags', 'q_view_count']
#unanswered_posts_questions_df = pd.DataFrame.from_records(results_list, columns=cols)
#unanswered_posts_questions_df



,q_id,q_title,q_body,accepted_answer_id,answer_count,q_creation_date,q_favorite_count,q_score,q_tags,q_view_count


In [12]:
# Determine data types for posts_questions_df
posts_questions_df.dtypes

q_id                                int64
q_title                            object
q_body                             object
accepted_answer_id                float64
answer_count                        int64
q_creation_date       datetime64[ns, UTC]
q_favorite_count                  float64
q_score                             int64
q_tags                             object
q_view_count                        int64
dtype: object

In [13]:
len(posts_questions_df)

1357455

In [14]:
# Determine number of answers
#posts_questions_df.groupby('answer_count').count()


In [15]:
print(posts_questions_df["answer_count"].max())
print(posts_questions_df["answer_count"].min())

26
0


In [16]:
# Bin answer count
answer_count_bins = [0,30]
answer_count_labels = ["good question (>0 answers)"]


In [17]:
#slice and bin answers
pd.cut(post_questions_df['answer_count'](labels=answer_count_labels).head()
                                        


SyntaxError: unexpected EOF while parsing (<ipython-input-17-387542cdedbf>, line 3)

In [18]:
 # Place the data series into a new column inside of the DataFrame
posts_questions_df["Answer Group"] = pd.cut(posts_questions_df["answer_count"], answer_count_bins, labels=answer_count_labels)
posts_questions_df

,q_id,q_title,q_body,accepted_answer_id,answer_count,q_creation_date,q_favorite_count,q_score,q_tags,q_view_count,Answer Group
0,68346458,Sort table by price,<p>I am having some problems sorting this <cod...,NaN,0,2021-07-12 11:18:33.270000+00:00,NaN,0,woocommerce|dokan,19,NaN
1,68128542,Webdriver IO with Mocha - Why do I not get a s...,<p>I am using webdriver IO with mocha.</p>\n<p...,NaN,0,2021-06-25 09:19:05.453000+00:00,NaN,0,mocha.js|webdriver-io,20,NaN
2,68339591,Single button click so that Angular kendo grid...,<p>Is it possible that an angular master detai...,NaN,0,2021-07-11 20:03:27.790000+00:00,NaN,0,angular|kendo-grid|master-detail,20,NaN
3,68225581,Multiple authors in Rmarkdown ioslides_present...,<p>I want to add two authors in my Rmarkdown p...,NaN,0,2021-07-02 12:53:34.307000+00:00,NaN,0,r-markdown,21,NaN
4,68387017,Add new plugin - cannot be added after `Amplif...,<p>I'm storing the user data locally using Dat...,NaN,0,2021-07-15 02:04:55.197000+00:00,NaN,0,amazon-web-services|aws-sdk|aws-amplify|amplif...,21,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1357450,67191286,"Crbug/1173575, non-JS module files deprecated....",<p>I just created new project and have run it ...,NaN,20,2021-04-21 07:37:43.460000+00:00,6.0,24,angular,128049,good question (>0 answers)
1357451,68387270,"Android Studio error ""Installed Build Tools re...",<p>I'm on Android Studio 4.2.2. I created a ne...,68397108.0,22,2021-07-15 02:45:41.303000+00:00,30.0,153,android|android-studio|android-emulator|androi...,93510,good question (>0 answers)
1357452,69021225,Resource linking fails on lStar,<p>I'm working on a React Native application. ...,69021606.0,23,2021-09-01 22:34:38.513000+00:00,2.0,64,android|react-native,9280,good question (>0 answers)
1357453,68191392,Password authentication is temporarily disable...,<p>I was using a username password for pushing...,68192700.0,25,2021-06-30 08:45:37.097000+00:00,50.0,237,git|github|token|github-api|git-pull,95253,good question (>0 answers)


## attempt model


In [19]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Create features
X = posts_questions_df.drop('Answer Group', axis=1)
X = pd.get_dummies(X)

# Create our target
y = posts_questions_df["Answer Group"]

X.head()

In [ ]:
X.describe()

In [ ]:
# Check the balance of our target values
y.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1,stratify=y)